In [21]:
from pyyoutube import Api
from youtube_api import YouTubeDataAPI
import datetime
import pandas as pd
from google.oauth2 import service_account
import pandas_gbq
import os

last_year = datetime.datetime.now() - datetime.timedelta(days=365)

#2 different APIs, as each one has a bug in a DIFFERENT functionality, thank god.
yt = YouTubeDataAPI(MY_API_KEY)
api = Api(api_key=MY_API_KEY)

In [58]:
baseURL = 'https://www.youtube.com/watch?v='
L = []
short_L = []

for phrase in ['meal prep', 'maquillaje natural']:
    for order_by in ['view_count', 'rating', 'relevance']:
        results = yt.search(phrase, published_after = last_year, max_results = 100, order_by=order_by)
        counter = 0
        for index, video in enumerate(results):
            counter += 1
            
            #POC
            if counter == 5:
                break
            video_info = api.get_video_by_id(video_id=video['video_id'])
            statistics = video_info.items[0].statistics
            snippet = video_info.items[0].snippet
            tags = None
            
            if snippet.tags != None:
                tags = ', '.join(snippet.tags).lower()
            
            #for BigQuery
            L.append({'phrase': phrase,
                      'order_by': order_by,
                      'url': '%s%s' % (baseURL, video['video_id']),
                      'viewCount': statistics.viewCount,
                      'likeCount': statistics.likeCount,
                      'publishedAt': datetime.datetime.strptime(snippet.publishedAt, '%Y-%m-%dT%H:%M:%SZ').strftime("%m/%d/%Y, %H:%M:%S"),
                      'title': snippet.title,
                      'description': snippet.description,
                      'thumbnail': snippet.thumbnails.default.url,
                      'channelTitle': snippet.channelTitle,
                      'tags': tags,
                      'index': index,
                      
                      #can be NULL
                      'defaultLanguage': snippet.defaultLanguage
                     })
            
            #for readable CSV
            short_L.append({
                
                'phrase': phrase,
                  'order_by': order_by,
                  'url': '%s%s' % (baseURL, video['video_id']),
                  'viewCount': statistics.viewCount,
                  'likeCount': statistics.likeCount,
            })

In [59]:
pd.DataFrame.from_dict(short_L).to_csv('data.csv', index = False)
pd.DataFrame.from_dict(L).to_csv('data_details.csv', index = False)

In [60]:


os.environ['APPLICATION_DEFAULT_CREDENTIALS'] = 'service_account.json'

#not really used
credentials = service_account.Credentials.from_service_account_file('service_account.json')
pandas_gbq.context.credentials = credentials

pandas_gbq.context.credentials = credentials
pandas_gbq.context.project = "youtube-scraper-aprende"


In [61]:
import google 

#not really used
google.auth.default()

//anaconda3/envs/scraping/lib/python3.8/site-packages/google/auth/_default.py:69: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. We recommend you rerun `gcloud auth application-default login` and make sure a quota project is added. Or you can use service accounts instead. For more information about service accounts, see https://cloud.google.com/docs/authentication/
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


(<google.oauth2.credentials.Credentials at 0x1232f0ac0>, None)

In [62]:
import pandas_gbq

#should be 0.13.0
pandas_gbq.__version__

'0.13.0'

In [63]:
data_to_write = pd.read_csv('data_details.csv')
data2 = pd.read_csv('data.csv')

from google.cloud import bigquery

#it works!
data_to_write.to_gbq('demo.demo3', credentials = credentials, if_exists = 'replace')

1it [00:04,  4.64s/it]


In [65]:
data_to_write

,phrase,order_by,url,viewCount,likeCount,publishedAt,title,description,thumbnail,channelTitle,tags,index,defaultLanguage
0,meal prep,view_count,https://www.youtube.com/watch?v=6pL_5zCHiT0,5590326,34589,"01/22/2020, 12:00:00",15 Efficient Ways to Meal Prep or Cook for a C...,Cooking for a crowd this weekend? Check out th...,https://i.ytimg.com/vi/6pL_5zCHiT0/default.jpg,Blossom,"blossom, diy, life hacks, hacks, diy hacks, bl...",0,NaN
1,meal prep,view_count,https://www.youtube.com/watch?v=F198LwSXoiM,2675123,109890,"01/15/2020, 19:31:43",Une journée dans mon assiette (x7)🌟MEAL PREP🌟 ...,▶︎LET'S BE FRIENDS💕💕\r\n I N S T A G R...,https://i.ytimg.com/vi/F198LwSXoiM/default.jpg,Ziya Inspiration,"lifestyle, meal prep, batch cooking, unejourne...",1,NaN
2,meal prep,view_count,https://www.youtube.com/watch?v=NOgPQvUWfHk,2616812,146632,"01/23/2020, 19:19:30",Meal prep on a budget » under €2 / $2 meals,» Get the PUL E-cookbook: http://bit.ly/PUL_Eb...,https://i.ytimg.com/vi/NOgPQvUWfHk/default.jpg,Pick Up Limes,"vegan, vegan recipes, vegan meals, vegan lunch...",2,en
3,meal prep,view_count,https://www.youtube.com/watch?v=dg1-CiVUU3I,1917342,103776,"12/31/2019, 16:42:01",MENÚ SEMANAL PARA DIETA KETO/PIERDE PESO RAPID...,MIS PRODUCTOS KETO FAVORITOS: https://shop.pe...,https://i.ytimg.com/vi/dg1-CiVUU3I/default.jpg,PINCHE NANCY,"keto, cetogenica, cetogenico, bajo en carbohid...",3,NaN
4,meal prep,rating,https://www.youtube.com/watch?v=tQ0xobgPJJY,92,10,"09/24/2020, 11:56:30",Grocery Haul | Meal Prep | Budget Friendly,Welcome to my channel. I do all things budgeti...,https://i.ytimg.com/vi/tQ0xobgPJJY/default.jpg,Jay B,"grocery haul, meal prep, budget friendly, low ...",0,NaN
5,meal prep,rating,https://www.youtube.com/watch?v=8Kspr7mFR2A,614,10,"03/30/2020, 13:00:04",#activitiesfortoddler How to keep your todd...,In this video i share how to keep your toddler...,https://i.ytimg.com/vi/8Kspr7mFR2A/default.jpg,Nene Folorunso,"home vlog, self isolation vlog, self isolation...",1,NaN
6,meal prep,rating,https://www.youtube.com/watch?v=2-WYRcRkbOc,65,3,"05/20/2020, 16:23:21",Meal Prep with David,OU Fit+Rec is here for you during this most ch...,https://i.ytimg.com/vi/2-WYRcRkbOc/default.jpg,OU Fitness and Recreation,NaN,2,NaN
7,meal prep,rating,https://www.youtube.com/watch?v=xILdH6FMl2s,171,11,"04/28/2020, 12:25:58",RAMADAN 2020 FREEZER MEAL PREP!!! | DAY 4,These are some freezer meals that I made for t...,https://i.ytimg.com/vi/xILdH6FMl2s/default.jpg,Fatima M,"accent, tag, baltimore, arab, muslim, hijab, h...",3,NaN
8,meal prep,relevance,https://www.youtube.com/watch?v=NOgPQvUWfHk,2616812,146632,"01/23/2020, 19:19:30",Meal prep on a budget » under €2 / $2 meals,» Get the PUL E-cookbook: http://bit.ly/PUL_Eb...,https://i.ytimg.com/vi/NOgPQvUWfHk/default.jpg,Pick Up Limes,"vegan, vegan recipes, vegan meals, vegan lunch...",0,en
9,meal prep,relevance,https://www.youtube.com/watch?v=0nK1trRuq7c,1049960,48976,"04/29/2020, 21:35:25",The Ultimate Minimalist Meal Prep,"Go to http://squarespace.com for a free trial,...",https://i.ytimg.com/vi/0nK1trRuq7c/default.jpg,Matt D'Avella,"meal prep, healthy meal prep, how to meal prep...",1,NaN


In [32]:
#an example video
video_info = api.get_video_by_id(video_id='SNZ6Qd58ixI')

//anaconda3/envs/scraping/lib/python3.8/site-packages/dataclasses_json/core.py:171: RuntimeWarning: `NoneType` object value of non-optional type blocked detected when decoding RegionRestriction.
  warnings.warn(f"`NoneType` object {warning}.", RuntimeWarning)


In [33]:
video_info.items[0].snippet.defaultLanguage